In [ ]:
import json
from pathlib import Path
from transformers import pipeline
_SWE_MODEL = "KBLab/wav2vec2-large-voxrex-swedish"
pipe = pipeline(model=_SWE_MODEL)
loc = Path("/home/joregan/christina-audio/")
for file in loc.glob("**/*.mp3"):
    output = pipe(str(file), chunk_length_s=10, return_timestamps="word")
    outname = loc / f"{file.stem}.json"
    with open(outname, "w") as of:
        json.dump(output, of)

In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
import json

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
model_id = "KBLab/kb-whisper-large"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, use_safetensors=True, cache_dir="cache"
)
model.to(device)
processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline("automatic-speech-recognition", model=model, tokenizer=processor.tokenizer, feature_extractor=processor.feature_extractor, torch_dtype=torch_dtype, device=device)

for file in loc.glob("**/*.mp3"):
    res = pipe(str(file), chunk_length_s=30, generate_kwargs={"task": "transcribe", "language": "sv"})
    outname = loc / f"{file.stem}.whisper.json"
    with open(outname, "w") as of:
       json.dump(res, of)